<a href="https://colab.research.google.com/github/nereaiscamu/ADA_2022/blob/main/latest_version_trying_coref.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install stanza; note that the prefix "!" is not needed if you are running in a terminal
!pip install stanza

# Import stanza
import stanza

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 691 kB 34.1 MB/s 
     |████████████████████████████████| 240 kB 77.0 MB/s 
  Created wheel for emoji: filename=emoji-2.2.0-py3-none-any.whl size=234926 sha256=4d9a48b19715d92060edbe736ec7a86ed41f1317b2c59ec9adbeb6d41be365b6
  Stored in directory: /root/.cache/pip/wheels/86/62/9e/a6b27a681abcde69970dbc0326ff51955f3beac72f15696984
Successfully built emoji


In [2]:
# Download the Stanford CoreNLP package with Stanza's installation command
# This'll take several minutes, depending on the network speed
corenlp_dir = './corenlp'
stanza.install_corenlp(dir=corenlp_dir)

# Set the CORENLP_HOME environment variable to point to the installation location
import os
os.environ["CORENLP_HOME"] = corenlp_dir

INFO:stanza:Installing CoreNLP package into ./corenlp


In [3]:
# Import client module
from stanza.server import CoreNLPClient

In [4]:
# Construct a CoreNLPClient with some basic annotators, a memory allocation of 4GB, and port number 9001
client = CoreNLPClient(
    annotators=['tokenize','ssplit', 'pos', 'lemma', 'ner', 'depparse', 'coref'], 
    memory='4G', 
    endpoint='http://localhost:9002',
    be_quiet=True)
print(client)

# Start the background server and wait for some time
# Note that in practice this is totally optional, as by default the server will be started when the first annotation is performed
client.start()
import time; time.sleep(10)



INFO:stanza:Writing properties to tmp file: corenlp_server-5b2ba03338c544d3.props
INFO:stanza:Starting server with command: java -Xmx4G -cp ./corenlp/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9002 -timeout 60000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-5b2ba03338c544d3.props -annotators tokenize,ssplit,pos,lemma,ner,depparse,coref -preload -outputFormat serialized


In [193]:
text = '''When his brother is killed in a robbery, paraplegic Marine Jake Sully decides to take his place in a mission on the distant world of Pandora. There he learns of greedy corporate figurehead Parker Selfridge's intentions of driving off the native humanoid "Na'vi" in order to mine for the precious material scattered throughout their rich woodland. In exchange for the spinal surgery that will fix his legs, Jake gathers knowledge, of the Indigenous Race and their Culture, for the cooperating military unit spearheaded by gung-ho Colonel Quaritch, while simultaneously attempting to infiltrate the Na'vi people with the use of an "avatar" identity. While Jake begins to bond with the native tribe and quickly falls in love with the beautiful alien Neytiri, the restless Colonel moves forward with his ruthless extermination tactics, forcing the soldier to take a stand - and fight back in an epic battle for the fate of Pandora.'''

In [194]:
#text = '''Jackson Maine, a famous country music singer privately battling an alcohol and drug addiction, plays a concert in California. His main support is Bobby, his manager and older half-brother. After the show, Jackson visits a drag bar where he witnesses a performance by Ally, a waitress and singer-songwriter. Jackson is amazed by her performance, and they spend the night speaking to each other, where Ally discloses to him the troubles she has faced in pursuing a professional music career. Jackson invites Ally to his next show. Despite her initial refusal she attends and, with Jackson's encouragement, sings on stage with him. Jackson invites Ally to go on tour with him, and they form a romantic relationship. In Arizona, Ally and Jackson visit the ranch where Jackson grew up and where his father is buried, only to discover that Bobby sold the land. Angered at his betrayal, Jackson punches Bobby, who subsequently quits as his manager. Before doing so, Bobby reveals that he did inform Jackson about the sale, but the latter was too inebriated to notice.'''


In [195]:
doc2 = client.annotate(text)
chains = doc2.corefChain
chain_dict=dict()
for index_chain,chain in enumerate(chains):
    chain_dict[index_chain]={}
    chain_dict[index_chain]['ref']=''
    chain_dict[index_chain]['mentions']=[{'mentionID':mention.mentionID,
                                          'mentionType':mention.mentionType,
                                          'number':mention.number,
                                          'gender':mention.gender,
                                          'animacy':mention.animacy,
                                          'beginIndex':mention.beginIndex,
                                          'endIndex':mention.endIndex,
                                          'headIndex':mention.headIndex,
                                          'sentenceIndex':mention.sentenceIndex,
                                          'position':mention.position,
                                          'ref':'',
                                          } for mention in chain.mention ]

clusters = []
sentences = []
names = []
beginIndex = []

for k,v in chain_dict.items():
    mentions=v['mentions']
    for mention in mentions:
        words_list = doc2.sentence[mention['sentenceIndex']].token[mention['beginIndex']:mention['endIndex']]
        for t in words_list:
          if (t.pos == 'NNP' or t.pos == 'PRP'):
            mention['ref']=t.word
            clusters.append(str(k))
            sentences.append(mention['sentenceIndex'])
            names.append(mention['ref'])
            beginIndex.append(mention['beginIndex'])


In [196]:
text_2

'When his brother is killed in a robbery, paraplegic Marine Jake Sully decides to take his place in a mission on the distant world of Pandora. There he learns of greedy corporate figurehead Parker Selfridge\'s intentions of driving off the native humanoid "Na\'vi" in order to mine for the precious material scattered throughout their rich woodland. In exchange for the spinal surgery that will fix his legs, Jake gathers knowledge, of the Indigenous Race and their Culture, for the cooperating military unit spearheaded by gung-ho Colonel Quaritch, while simultaneously attempting to infiltrate the Na\'vi people with the use of an "avatar" identity. While Jake begins to bond with the native tribe and quickly falls in love with the beautiful alien Neytiri, the restless Colonel moves forward with his ruthless extermination tactics, forcing the soldier to take a stand - and fight back in an epic battle for the fate of Pandora.'

In [197]:
import pandas as pd
clus_table = pd.DataFrame()
clus_table = clus_table.assign(cluster =clusters)
clus_table = clus_table.assign(sentence =sentences)
clus_table = clus_table.assign(name =names)
#clus_table = clus_table.assign(beginIndex =beginIndex)
clus_table

,cluster,sentence,name
0,0,3,Pandora
1,0,0,Pandora
2,1,2,Jake
3,1,1,he
4,1,3,Jake
5,1,3,Colonel
6,1,0,Marine
7,1,0,Jake
8,1,0,Sully


In [198]:
stanza.download('en') # download English model


INFO:stanza:Downloading default packages for language: en (English) ...
INFO:stanza:File exists: /root/stanza_resources/en/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources.


In [199]:
nlp = stanza.Pipeline(lang='en', processors='tokenize, pos, lemma, depparse, ner')


INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | combined  |
| pos       | combined  |
| lemma     | combined  |
| depparse  | combined  |
| ner       | ontonotes |

INFO:stanza:Use device: gpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: depparse
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!


In [200]:
doc = nlp(text)

In [201]:
def get_characters(doc):
    characters = []
    characters_name = []
    for sent in doc.sentences:
        for word in sent.ents:
            if word.type == 'PERSON' and word.text not in characters:
                characters.append([word.text])
                characters_name.append([word.text.split(' ')[0]])
    characters = list(np.unique(characters))
    characters_name = list(np.unique(characters_name))
    return characters, characters_name

In [202]:
import numpy as np

get_characters(doc)

(['Jake', 'Jake Sully', 'Neytiri', "Parker Selfridge's", 'Quaritch'],
 ['Jake', 'Neytiri', 'Parker', 'Quaritch'])

In [203]:
chars = get_characters(doc)[1]
clus_table["char"] = np.where(clus_table["name"].isin(chars), clus_table["name"], 0)

clus2 = clus_table.loc[clus_table['char']!=0]
clus2 = clus2.groupby('cluster')['char'].unique()
clus3 = pd.merge(clus_table, clus2, on = 'cluster')  
char_clean = []
for i in clus3['char_y']:
  char_clean.append(i[0])
clus3['character'] = char_clean
clus3 = clus3.drop(columns = ['cluster','char_x', 'char_y'])
char_table = clus3

In [204]:
text

'When his brother is killed in a robbery, paraplegic Marine Jake Sully decides to take his place in a mission on the distant world of Pandora. There he learns of greedy corporate figurehead Parker Selfridge\'s intentions of driving off the native humanoid "Na\'vi" in order to mine for the precious material scattered throughout their rich woodland. In exchange for the spinal surgery that will fix his legs, Jake gathers knowledge, of the Indigenous Race and their Culture, for the cooperating military unit spearheaded by gung-ho Colonel Quaritch, while simultaneously attempting to infiltrate the Na\'vi people with the use of an "avatar" identity. While Jake begins to bond with the native tribe and quickly falls in love with the beautiful alien Neytiri, the restless Colonel moves forward with his ruthless extermination tactics, forcing the soldier to take a stand - and fight back in an epic battle for the fate of Pandora.'

In [205]:
char_table

,sentence,name,character
0,2,Jake,Jake
1,1,he,Jake
2,3,Jake,Jake
3,3,Colonel,Jake
4,0,Marine,Jake
5,0,Jake,Jake
6,0,Sully,Jake


In [206]:
names = []
sent_ix = []
charss = []
for i,sent in enumerate(doc.sentences):
  nouns = [w.text for w in sent.words if (w.pos == "PROPN" and w.text in chars)]
  for n in nouns:
    sent_ix.append(i)
    names.append(n)
    charss.append(n)
char_table2 = pd.DataFrame()
char_table2['sentence'] = sent_ix
char_table2['name'] = names
char_table2['character'] = charss
char_table_all = pd.concat([char_table, char_table2],  ignore_index=True)
char_table_all = char_table_all.sort_values('sentence').reset_index()
char_table_all = char_table_all.drop(columns = ['index']).drop_duplicates()
char_table_all

,sentence,name,character
0,0,Marine,Jake
1,0,Jake,Jake
2,0,Sully,Jake
4,1,he,Jake
5,1,Parker,Parker
6,2,Jake,Jake
8,2,Quaritch,Quaritch
9,3,Jake,Jake
10,3,Colonel,Jake
12,3,Neytiri,Neytiri


In [207]:
def recursive_find_adjs(root, sentence):
    children = [w for w in sentence.words if w.head == root.id]
    if not children:
        pass 
    filtered_child = [w for w in children if (w.deprel == "conj" or w.deprel == "appos" 
                                              or w.deprel == "nmod"
                                              or w.deprel == "amod") and (w.pos == "ADJ"or w.pos == 'NOUN')] 
    results = [w for w in filtered_child if not any(sub.head == w.id and sub.upos == "NOUN" for sub in sentence.words)]
    for w in children:
        results += recursive_find_adjs(w, sentence)
    return results

In [218]:
def char_attributes(doc):
    sentences = []
    names = []
    attributes = []
    for ix, sent in enumerate(doc.sentences):

      nouns = [w for w in sent.words if (w.pos == "PROPN" or w.pos == "PRON")]
      
      name_list = list(char_table_all.loc[char_table_all['sentence']==ix]['name'])
      for noun in nouns:
        if noun.text in name_list:
            adj0 = sent.words[noun.head-1] #adjective directly related
            adjs = [adj0] + recursive_find_adjs(adj0, sent) if adj0.pos == "ADJ" or adj0.pos == "NOUN" else []
            '''deps = [w for w in sent.words if (w.head == noun.id and (w.upos == 'NOUN' or w.upos == 'ADJ')) ]
            for i in range(len(deps)):
              adj0 = deps[i]
              print(adj0.text)
              print('\n')
              adjs = [adj0] + recursive_find_adjs(adj0, sent) if adj0.pos == "ADJ" or adj0.pos == "NOUN" else [] '''
            mod_adjs = [w for w in sent.words if w.head == noun.id and (w.pos == "ADJ" or w.pos == 'NOUN')]
              # This should only be one element because conjunctions are hierarchical
            if mod_adjs:
              mod_adj = mod_adjs[0]
              adjs.extend([mod_adj] + recursive_find_adjs(mod_adj, sent))
            if adjs:
              unique_adjs = []
              unique_ids = set()
              for adj in adjs:
                  if adj.id not in unique_ids:
                      unique_adjs.append(adj)
                      unique_ids.add(adj.id)
              sentences.append(ix)
              names.append(noun.text)
              attributes.append(" ".join([adj.text for adj in unique_adjs]))
    char_attributes = pd.DataFrame()
    char_attributes['Sentence'] = sentences
    char_attributes['Character Names'] = names
    char_attributes['Character Attributes'] = attributes
    char_attributes['Total Attributes'] = char_attributes.groupby(['Sentence','Character Names'])['Character Attributes'].transform(lambda x: ' '.join(x))
    char_attributes= char_attributes[['Sentence', 'Character Names','Total Attributes']]
    return (char_attributes.drop_duplicates().reset_index())




,index,Sentence,Character Names,Total Attributes
0,0,0,Marine,paraplegic
1,1,1,Parker,figurehead greedy corporate
2,2,2,Quaritch,gung
3,3,3,Neytiri,love beautiful alien
4,4,3,Colonel,restless


In [220]:
result_attr = char_attributes(doc)
char_table_all.merge(result_attr, left_on = ['sentence','name'], right_on = ['Sentence','Character Names'])


,sentence,name,character,index,Sentence,Character Names,Total Attributes
0,0,Marine,Jake,0,0,Marine,paraplegic
1,1,Parker,Parker,1,1,Parker,figurehead greedy corporate
2,2,Quaritch,Quaritch,2,2,Quaritch,gung
3,3,Colonel,Jake,4,3,Colonel,restless
4,3,Neytiri,Neytiri,3,3,Neytiri,love beautiful alien


In [211]:
def char_attributes(doc):
    names = []
    prons = []
    attributes = []
    attributes_pr = []
    for ix, sent in enumerate(doc.sentences):
        nouns = [w for w in sent.words if w.pos == "PROPN"]
        pronouns = [w for w in sent.words if w.pos == "PRON"]
        name_list = list(char_table_all.loc[char_table_all['sentence']==ix]['name'])
        for noun in nouns:
            if noun.text in name_list:
                              # Find constructions in the form of "The car is beautiful"
                # In this scenario, the adjective is the parent of the noun
                adj0 = sent.words[noun.head-1] #adjective directly related
                adjs = [adj0] + recursive_find_adjs(adj0, sent) if adj0.pos == "ADJ" or adj0.pos == "NOUN" else []
                #The recursive function finds adjectives related to the first one found,
                #and hence also linked to the target noun
                mod_adjs = [w for w in sent.words if w.head == noun.id and (w.pos == "ADJ" or w.pos == 'NOUN')]
                # This should only be one element because conjunctions are hierarchical
                if mod_adjs:
                    mod_adj = mod_adjs[0]
                    adjs.extend([mod_adj] + recursive_find_adjs(mod_adj, sent))
                if adjs:
                    unique_adjs = []
                    unique_ids = set()
                    for adj in adjs:
                        if adj.id not in unique_ids:
                            unique_adjs.append(adj)
                            unique_ids.add(adj.id)
                    names.append(noun.text)
                    attributes.append(" ".join([adj.text for adj in unique_adjs]))
        '''for pr in pronouns:
            if pr.text in name_list:
                adj0 = sent.words[pr.head-1] #adjective directly related
                adjs = [adj0] + recursive_find_adjs(adj0, sent) if adj0.pos == "ADJ" or adj0.pos == "NOUN" else []
                #The recursive function finds adjectives related to the first one found,
                #and hence also linked to the target noun
                mod_adjs = [w for w in sent.words if w.head == pr.id and (w.pos == "ADJ" or w.pos == 'NOUN')]
                # This should only be one element because conjunctions are hierarchical
                if mod_adjs:
                    mod_adj = mod_adjs[0]
                    adjs.extend([mod_adj] + recursive_find_adjs(mod_adj, sent))
                if adjs:
                    unique_adjs = []
                    unique_ids = set()
                    for adj in adjs:
                        if adj.id not in unique_ids:
                            unique_adjs.append(adj)
                            unique_ids.add(adj.id)
                    prons.append(pr.text)
                    attributes_pr.append(" ".join([adj.text for adj in unique_adjs]))
                    print(attributes_pr)'''
    char_attributes = pd.DataFrame()
    char_attributes['Character Names'] = names
    char_attributes['Character Attributes'] = attributes
    char_attributes['Total Attributes'] = char_attributes.groupby('Character Names')['Character Attributes'].transform(lambda x: ' '.join(x))
    char_attributes= char_attributes[['Character Names','Total Attributes']]
    return (char_attributes.drop_duplicates().reset_index())

char_attributes(doc)

,index,Character Names,Total Attributes
0,0,Marine,paraplegic
1,1,Parker,figurehead greedy corporate
2,2,Quaritch,gung
3,3,Neytiri,love beautiful alien
4,4,Colonel,restless


In [212]:
def char_attributes(doc):
    names = []
    attributes = []
    
    for sent in doc.sentences:
        nouns = [w for w in sent.words if w.pos == "PROPN"]
        for noun in nouns:
            if noun.text in get_characters(doc)[1]:
                # Find constructions in the form of "The car is beautiful"
                # In this scenario, the adjective is the parent of the noun
                adj0 = sent.words[noun.head-1] #adjective directly related
                adjs = [adj0] + recursive_find_adjs(adj0, sent) if adj0.pos == "ADJ" or adj0.pos == "NOUN" else []
                #The recursive function finds adjectives related to the first one found,
                #and hence also linked to the target noun
                mod_adjs = [w for w in sent.words if w.head == noun.id and (w.pos == "ADJ" or w.pos == 'NOUN')]
                # This should only be one element because conjunctions are hierarchical
                if mod_adjs:
                    mod_adj = mod_adjs[0]
                    adjs.extend([mod_adj] + recursive_find_adjs(mod_adj, sent))
                if adjs:
                    unique_adjs = []
                    unique_ids = set()
                    for adj in adjs:
                        if adj.id not in unique_ids:
                            unique_adjs.append(adj)
                            unique_ids.add(adj.id)
                    names.append(noun.text)
                    attributes.append(" ".join([adj.text for adj in unique_adjs]))
    char_attributes = pd.DataFrame()
    char_attributes['Character Names'] = names
    char_attributes['Character Attributes'] = attributes
    char_attributes['Total Attributes'] = char_attributes.groupby('Character Names')['Character Attributes'].transform(lambda x: ' '.join(x))
    char_attributes= char_attributes[['Character Names','Total Attributes']]
    return (char_attributes.drop_duplicates().reset_index())

In [213]:
char_attributes(doc)

,index,Character Names,Total Attributes
0,0,Parker,figurehead greedy corporate
1,1,Quaritch,gung
2,2,Neytiri,love beautiful alien


In [ ]:
# Convert dependency tree formats
sent = doc2.sentence[1]
print(sent.enhancedPlusPlusDependencies.edge[0])
print ( sent.token[3])


source: 3
target: 1
dep: "nmod:poss"
isExtra: false
sourceCopy: 0
targetCopy: 0
language: UniversalEnglish

word: "is"
pos: "VBZ"
value: "is"
before: " "
after: " "
originalText: "is"
ner: "O"
lemma: "be"
beginChar: 143
endChar: 145
utterance: 0
speaker: "PER0"
tokenBeginIndex: 25
tokenEndIndex: 26
hasXmlContext: false
isNewline: false
coarseNER: "O"
fineGrainedNER: "O"
nerLabelProbs: "O=0.9998249819540405"



In [ ]:
from collections import defaultdict
from stanza.server import CoreNLPClient

text = "Barack Obama was born in Hawaii. In 2008 he became the president."

doc = client.annotate(text)

animacy = defaultdict(dict)
for x in doc.corefChain:
    for y in x.mention:
        print(y.animacy)
        for i in range(y.beginIndex, y.endIndex):
            animacy[y.sentenceIndex][i] = True
            print(y.sentenceIndex, i)

for sent_idx, sent in enumerate(doc.sentence):
    print("[Sentence {}]".format(sent_idx+1))
    for t_idx, token in enumerate(sent.token):
        animate = animacy[sent_idx].get(t_idx, False)
        print("{:12s}\t{:12s}\t{:6s}\t{:20s}\t{}".format(token.word, token.lemma, token.pos, token.ner, animate))
    print("")

ANIMATE
0 0
0 1
ANIMATE
1 2
[Sentence 1]
Barack      	Barack      	NNP   	PERSON              	True
Obama       	Obama       	NNP   	PERSON              	True
was         	be          	VBD   	O                   	False
born        	bear        	VBN   	O                   	False
in          	in          	IN    	O                   	False
Hawaii      	Hawaii      	NNP   	STATE_OR_PROVINCE   	False
.           	.           	.     	O                   	False

[Sentence 2]
In          	in          	IN    	O                   	False
2008        	2008        	CD    	DATE                	False
he          	he          	PRP   	O                   	True
became      	become      	VBD   	O                   	False
the         	the         	DT    	O                   	False
president   	president   	NN    	TITLE               	False
.           	.           	.     	O                   	False



In [ ]:
enhancedPlusPlusDependencies

In [ ]:
from stanza.server import CoreNLPClient
client = CoreNLPClient(
        annotators=['tokenize','ssplit','pos','lemma','ner', 'parse', 'openie','depparse','coref'],
        timeout=30000,
        memory='4G')

test ="A man and a woman came into the store."
matches = client.tregex(text, 'S')
print(matches['sentences'][0]['0']['match'])

INFO:stanza:Writing properties to tmp file: corenlp_server-2b4604e86dc04f69.props
INFO:stanza:Starting server with command: java -Xmx4G -cp ./corenlp/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 30000 -threads 5 -maxCharLength 100000 -quiet False -serverProperties corenlp_server-2b4604e86dc04f69.props -annotators tokenize,ssplit,pos,lemma,ner,parse,openie,depparse,coref -preload -outputFormat serialized


KeyboardInterrupt: ignored

In [ ]:
# submit the request to the server
sentence = doc2.sentence[1]
print('dependency parse of first sentence')
dependency_parse = sentence.enhancedPlusPlusDependencies


for t in sentence.token:
  print(t.word)

print('\n')
print(sentence.token[dependency_parse.node[4].index].word)
#print(dependency_parse.node)

print('\n')


for i in dependency_parse.edge:
  if i.target == dependency_parse.node[4].index:
    print(sentence.token[i.target-1].word)
    print(sentence.token[i.source-1].word)
    print(i.dep)


dependency parse of first sentence
His
main
support
is
Bobby
,
his
manager
and
older
half
-
brother
.


Bobby


is
Bobby
cop


In [ ]:
!pip install spacy
!pip install pip install corenlp-vdep


import spacy
from spacy import displacy

from corenlp_dtree_visualizer.converters import _corenlp_dep_tree_to_spacy_dep_tree



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Convert dependency tree formats
sent = doc2.sentence[0]

# Visualize with Spacy
nlp = spacy.load("en_core_web_sm")
displacy.render(tree, style="dep", manual=True)

# could also save to a file
# svg = displacy.render(tree, style="dep", manual=True)
# with open('tmp.svg', 'w', encoding='utf-8') as fw:
    # fw.write(svg)

TypeError: ignored